Need to clean the code
1. dir 일원화
2. 데이터 형식의 차이에 따른 if문 사용
3. 이미 전처리 된 데이터가 있는 경우 제외
4. light cat 수정 필요
5. integrate시 존재하는 파일에 추가

### Import Module

In [1]:
from glob import glob
import os, sys
from os.path import join, dirname

import datetime, time
import csv
from glob import glob
import chardet
import numpy as np
import pandas as pd

from tqdm import notebook

### Load Data

In [2]:
root_dir = '../Data'

EMA_dir = root_dir + '/KOR_EMA_Data'
Collar_dir = root_dir + '/KOR_Collar_Data'
preprocessed_dir = root_dir + '/Preprocessed'

pass_list = glob(preprocessed_dir + '/*.CSV')
EMA_list = glob(EMA_dir + '/*.xlsx')
Collar_list = glob(Collar_dir + '/*.CSV')
preprocessed_data_name = preprocessed_dir + '/preprocessed_data(with_4_korean).csv'

In [3]:
ID_list = []
target_data_list = []
feature_data_list = []


for path in EMA_list:
    temp = path.split('\\')
    temp = temp[1].split('_')
    ID_list.append(temp[0])

for name in ID_list:
    tmp_target = preprocessed_dir + '\\target_' + name + '.csv'
    tmp_feature = preprocessed_dir + '\\feature_' + name + '.csv'
    target_data_list.append(tmp_target)
    feature_data_list.append(tmp_feature)

### Preprocess Time stamp

In [4]:
def convert24(time_stamp): 
    
    time_stamp = str(pd.to_datetime(time_stamp, infer_datetime_format=True))
    # Checking if last two elements of time 
    # is AM and first two elements are 12 
    
    if time_stamp[-2:] == "AM" or time_stamp[-2:] == "PM":
        if time_stamp[-2:] == "AM" and time_stamp[11:13] == "12": 
            return time_stamp[:11] + "00" + time_stamp[13:-2] 

        # remove the AM  
        elif time_stamp[-2:] == "AM": 
            return time_stamp[:-2] 

        # Checking if last two elements of time 
        # is PM and first two elements are 12 
        elif time_stamp[-2:] == "PM" and time_stamp[11:13] == "12": 
            return time_stamp[:-2] 

        else: 

            # add 12 to hours and remove PM 
            return time_stamp[:11] + str(int(time_stamp[11:13]) + 12) + time_stamp[13:-2]
    
    else: 
        return time_stamp

### Preprocess Target Data

In [5]:
## Dictionary for response types
res = {"0" : "Playing",
       "1" : "Talking",
       "2" : "Petting",
       "3" : "TV / Radio",
       "4" : "Eating / Cooking",
       "5" : "Moved It",
       "6" : "None of the above",
       "7" : "Other"}

## EMA Data collumn name
EMA_col_name = ["Start Date","End Date","Time Scheduled","Duration (in seconds)","Finished","Participant ID","Location - Lat",
                "Location - Long","Survey","Question 1","Question 2","Question 3","Question 4.1","Question 4.2","Question 4.3",
                "Question 4.4","Question 4.5","Question 4.6","Question 4.7","Question 4.8","Question 5","Question 6","Question 7"]

for x in range(len(EMA_list)):
    
    EMA = pd.read_excel(EMA_list[x], engine ='openpyxl')
    EMA = EMA.to_records(index=False)
    
    ## Row number in will-be-created file
    rowid = 1
    prox = 0
    
    ## Write a csv file
    if target_data_list[x] not in pass_list:
        with open(target_data_list[x], 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["RowID", "Modality_cat", "Modality", "Proximity", "Start Window", "End Window"])


            for i in range(len(EMA)):
                if EMA[i]["Question 1"] == "Yes":  # Check if the participant has interacted

                    interation_type = []
                    prox = 0  # prox reset

                    ## Check the number of interaction types
                    for j in range(12, 20):
                        if EMA[i][j] == "Yes":
                            interation_type.append(j-12)

                    for k in range(len(interation_type)):
                        end_time = convert24(EMA[i]["Start Date"])
                        start_time = pd.to_datetime(end_time, infer_datetime_format=True) - pd.Timedelta(pd.offsets.Minute(15))
                        writer.writerow([rowid, interation_type[k], res[str(interation_type[k])], EMA[i]["Question 6"], start_time, end_time]) # Write data one (row) by one..
                        rowid += 1

### Preprocess Feature Data

In [7]:
Collar_list

['../Data/KOR_Collar_Data\\KORp1_Collar_data.CSV',
 '../Data/KOR_Collar_Data\\KORp2_Collar_data.CSV',
 '../Data/KOR_Collar_Data\\KORp3_Collar_data.CSV',
 '../Data/KOR_Collar_Data\\KORp4_Collar_data.CSV']

In [8]:
collar_col_name = ['awake', 'sound_val', 'sound_cat', 'light_val', 'accel_x', 'accel_y', 'accel_z', 'orient', 'motion_detect', 'sound_detect)', 'Realtime']

for i in range(len(Collar_list)):
    
    if feature_data_list[i] not in pass_list:
        print(ID_list[i])

        collar_data = pd.read_csv(Collar_list[i])           # load ith collar_data
        collar_data = collar_data[collar_col_name]          # 
    #     idx = collar_data[collar_data["Realtime"] == "Realtime"].index
        collar_data.drop(collar_data[collar_data["Realtime"] == "Realtime"].index,inplace=True)
        collar_data = collar_data.to_records(index=False)

        target_data = pd.read_csv(target_data_list[i])
        target_data = target_data.to_records(index=False)

        with open(feature_data_list[i], 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['RowID', 'Real_time', 'awake', 'sound_val', 'sound_cat', 'light_val', 'accel_x', 'accel_y', 'accel_z', 'orient', 'motion_detect', 'sound_detect)'])

            start = 0

            for row in notebook.tqdm(range(len(target_data))):
                for j in range(start, len(collar_data)):
                    if convert24(collar_data[j]["Realtime"]) > convert24(target_data[row]['End Window']):
                        break

                    if convert24(collar_data[j]["Realtime"]) < convert24(target_data[row]['Start Window']):
                        start += 1

                    if convert24(collar_data[j]["Realtime"]) >= convert24(target_data[row]['Start Window']) and convert24(collar_data[j]["Realtime"]) <= convert24(target_data[row]['End Window']):
                        writer.writerow([target_data[row]['RowID'], collar_data[j]['Realtime'], collar_data[j]['awake'], collar_data[j]['sound_val'], collar_data[j]['sound_cat'], collar_data[j]['light_val'], collar_data[j]['accel_x'], collar_data[j]['accel_y'], collar_data[j]['accel_z'], collar_data[j]['orient'], collar_data[j]['motion_detect'], collar_data[j]['sound_detect)']])

KORp3


C:\Users\sckim\.conda\envs\grad\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0,1,3,4,5,7,8,9,10,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



KORp4


In [9]:
for i in range(len(ID_list)):
    rowid = []
    
    feature_data = pd.read_csv(feature_data_list[i])
    feature_data = feature_data.to_records(index=False)
    target_data = pd.read_csv(target_data_list[i])
    
    for j in range(len(feature_data)):
        if feature_data[j]['RowID'] not in rowid:
            rowid.append(feature_data[j]['RowID'])
            
    print(ID_list[i], ' ', len(rowid) / len(target_data))
    print(len(rowid), len(feature_data))
    print("\n")

KORp1   0.40425531914893614
19 47104


KORp2   0.37254901960784315
95 199641


KORp3   0.6538461538461539
17 37857


KORp4   0.8181818181818182
99 153037




### Integrate Data

In [5]:
def diff_create(df, accel_k, diff_k):
    df[diff_k]=df[accel_k]
    for i in range(len(df)):
        if i==0:
            continue
        elif df['RowID'][i]!=df['RowID'][i-1]:
            df[diff_k][i]=0
            continue
        else:
            df[diff_k][i]-=df[accel_k][i-1]
    df[diff_k][0]=0

def chord_arc_create (df,diff_x,diff_y,diff_z) :
    df['chord'] = np.sqrt(df[diff_x]**2+df[diff_y]**2+df[diff_z]**2)
    df['arc'] = 2*np.arcsin(df['chord']/2)

In [8]:
file_list = os.listdir(preprocessed_dir)
target = []
feature = []

for path in file_list:
    temp = path.split('_')
    if temp[0] == 'feature':
        temp_dir = preprocessed_dir + "/" + path
        feature.append(temp_dir)
    elif temp[0] == 'target':
        temp_dir = preprocessed_dir + "/" + path
        target.append(temp_dir)
    else:
        pass
    
print(feature)

['../Data/Preprocessed/feature_Bree.csv', '../Data/Preprocessed/feature_Daisey.csv', '../Data/Preprocessed/feature_Grace.csv', '../Data/Preprocessed/feature_KORp1.csv', '../Data/Preprocessed/feature_KORp2.csv', '../Data/Preprocessed/feature_KORp3.csv', '../Data/Preprocessed/feature_KORp4.csv', '../Data/Preprocessed/feature_Mickey.csv']


In [12]:
# def scaler(num):
#     max_num = 4000
#     min_num = 0
    
#     if (num - min_num) / (max_num - min_num) * 1024 > 1024:
#         return 1
#     else: 
#         return round((num - min_num) / (max_num - min_num) * 1024)

In [12]:
with open(preprocessed_data_name, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['RowID', 'Real_time', 'awake', 'sound_val', 'sound_cat', 'light_val', 'accel_x', 'accel_y', 'accel_z', 'orient', 'chord', 'arc', 'motion_detect', 'sound_detect)', "Modality_cat", "Modality"])
    
    for i in range(len(target)):
        rowid = []

        feature_data = pd.read_csv(feature[i])
        target_data = pd.read_csv(target[i])
        target_data = target_data.to_records(index=False)

        diff_create(feature_data,'accel_x','diff_x')
        diff_create(feature_data,'accel_y','diff_y')
        diff_create(feature_data,'accel_z','diff_z')
        chord_arc_create (feature_data,'diff_x','diff_y','diff_z')

        feature_data = feature_data.to_records(index=False)
        
        temp = feature[i].split("/")
        temp = temp[3].split(".")
        temp = temp[0].split("_")
        
        if temp[1][:3] == 'KOR':
            print(temp[1])
            for j in range(len(feature_data)):
                idx = np.where(target_data['RowID'] == feature_data[j]['RowID'])
                new_RowID = '1' + '_' + str(i) + '_' + str(feature_data[j]['RowID'])
                writer.writerow([new_RowID, feature_data[j]['Real_time'], feature_data[j]['awake'], feature_data[j]['sound_val'], feature_data[j]['sound_cat'], feature_data[j]['light_val'], feature_data[j]['accel_x'], feature_data[j]['accel_y'], feature_data[j]['accel_z'], feature_data[j]['orient'], feature_data[j]['chord'], feature_data[j]['arc'], feature_data[j]['motion_detect'], feature_data[j]['sound_detect)'], target_data[idx[0][0]]["Modality_cat"], target_data[idx[0][0]]["Modality"]])
        
        else:
            print(temp[1])
            for j in range(len(feature_data)):
                idx = np.where(target_data['RowID'] == feature_data[j]['RowID'])
                new_RowID = '0' + '_' + str(i) + '_' + str(feature_data[j]['RowID'])
                writer.writerow([new_RowID, feature_data[j]['Real_time'], feature_data[j]['awake'], feature_data[j]['sound_val'], feature_data[j]['sound_cat'], feature_data[j]['light_val'], feature_data[j]['accel_x'], feature_data[j]['accel_y'], feature_data[j]['accel_z'], feature_data[j]['orient'], feature_data[j]['chord'], feature_data[j]['arc'], feature_data[j]['motion_detect'], feature_data[j]['sound_detect)'], target_data[idx[0][0]]["Modality_cat"], target_data[idx[0][0]]["Modality"]])            

<ipython-input-5-4c135b961834>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[diff_k][i]-=df[accel_k][i-1]
<ipython-input-5-4c135b961834>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[diff_k][i]=0
<ipython-input-5-4c135b961834>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[diff_k][0]=0


Bree
Daisey
Grace
KORp1
KORp2
KORp3
KORp4
Mickey
